In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import feature_functions as ff

In [2]:
df=pd.DataFrame()

In [3]:
df['Stock']= ["GREATEC",
        "UWC"]

In [4]:
data=ff.obtain_stocks_data(df['Stock'])

In [5]:
all_combine_data=list(data.keys())

In [6]:
all_combine_data

['GREATEC', 'UWC']

In [8]:
data=list(data.values())[0]

In [9]:
data

,status,c,d,h,l,o,v
0,authorized. data since: 13-06-2019,0.3700,2019-06-13,0.3900,0.3175,0.3175,385442800.0
1,authorized. data since: 13-06-2019,0.4050,2019-06-14,0.4275,0.3775,0.3775,224210400.0
2,authorized. data since: 13-06-2019,0.4500,2019-06-17,0.4500,0.4150,0.4175,155926200.0
3,authorized. data since: 13-06-2019,0.4950,2019-06-18,0.5000,0.4450,0.4625,191351800.0
4,authorized. data since: 13-06-2019,0.4825,2019-06-19,0.5050,0.4725,0.4975,112346600.0
...,...,...,...,...,...,...,...
442,authorized. data since: 13-06-2019,5.4200,2021-03-29,5.4400,5.3200,5.3300,830100.0
443,authorized. data since: 13-06-2019,5.3100,2021-03-30,5.4500,5.2400,5.4300,1437300.0
444,authorized. data since: 13-06-2019,5.3200,2021-03-31,5.3600,5.2600,5.2600,1304400.0
445,authorized. data since: 13-06-2019,5.4200,2021-04-01,5.4200,5.3000,5.3500,2240700.0


In [10]:
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# makes plots bigger
plt.rcParams["figure.figsize"] = (20,8)

#Tuning OB and OS level
ob_cross=85
os_cross=35

df= pd.DataFrame.from_dict(data)
df= df[['c','d','h','l','o','v']]
df['d'] = pd.to_datetime(df['d'], dayfirst= True )
#identify MA50
n=50
ma50=[] 

for i in range(len(df["c"])-(n-1)):
    ma = round(sum(df["c"][i:i+(n)])/n,4)
    #print(ma)
    ma50.append(ma)

df["MA50"] = pd.Series(ma50)
df["MA50"] = df["MA50"].shift(n-1)

#filter uptrend price above ma50
# Calculating the gap of both MA
df["MAgap"] = df["c"] -  df["MA50"] 

df.set_index('d',inplace=True,drop=True)
df.index.name = None

#create signal for uptrend price above ma50 
#result not accurate for stochastic when just filter uptrend only
df["Uptrend"]=df['MAgap']>0
Uptrend=df["Uptrend"]
Uptrend_signal=Uptrend[Uptrend==True]
#signal stochastic
#Create the "L14" column in the DataFrame
df['L14'] = df['l'].rolling(window=14).min()

#Create the "H14" column in the DataFrame
df['H14'] = df['h'].rolling(window=14).max()

#Create the "%K" column in the DataFrame
df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

#Create the "%D" column in the DataFrame
df['%D'] = df['%K'].rolling(window=3).mean()

#Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
#A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

#Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
#A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

overbought =df['Sell OB']
oversold =df['Buy OS']
date_os=list(oversold[oversold == True].index)
date_ob=list(overbought[overbought == True].index)

# transaction recording


# transaction recording
buy = []
sell = []
sdate = []
bdate = []
wr = []
pct = []
vol = []
ope =[]
high = []
low = []


for n,i in enumerate(date_os):
    #print(n,"Buy:", i, "Price:", df['c'].loc[i])
    buy.append(df['c'].loc[i])
    vol.append(df['v'].loc[i])
    ope.append(df['o'].loc[i])
    high.append(df['h'].loc[i])
    low.append(df['l'].loc[i])

    bdate.append(i)
    r=[]
    #print("before", len(r))
    for m,k in enumerate(date_ob):

        if k>i:
            r.append(m)
            #print('after',len(r))
            if len(r) ==1:
                #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                sell.append(df['c'].loc[k])
                sdate.append(k)
                pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                wr =pct_change>0

                #if pct_change >0:
                    #print("1")
                #else:
                    #print('0')

            elif len(r)>1:
                pass       
    #print('........')
#print('Number of trade:',len(date_os))

diff =len(buy)-len(sell)
#print('difference:',diff)

# Checking for final transaction, make sure it is sell, as we want to close the position
if len(buy) == len(sell):
    print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
    print("Position closed")

elif len(buy) != len(sell):
    #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
    del(buy[-diff:])
    del(bdate[-diff:])
    del(ope[-diff:])
    del(high[-diff:])
    del(vol[-diff:])
    del(low[-diff:])
    #print("Position closed")
dct = {
    "Buy_Date": bdate,
    "Buy": buy,
    "Sell": sell,
    "Sell_Date": sdate,
    "Open":ope,
    'Volume': vol,
    "Low": low,
    "high": high,
    "Pct_change": pct    
}

trans = pd.DataFrame(dct)

wr = []
for i in range(len(trans)):
    if trans["Pct_change"][i] > 0:
        wr.append(1)
    else:
        wr.append(0)
trans["wr"] = pd.Series(wr)

#Name_stock = []
#for s in range(len(trans)):
    #Name_stock.append(symbol)


#trans['Stock'] = pd.Series(Name_stock)
trans

,Buy_Date,Buy,Sell,Sell_Date,Open,Volume,Low,high,Pct_change,wr
0,2019-11-25,0.960,1.095,2019-12-09,0.960,3698600.0,0.945,0.965,14.062500,1
1,2019-11-27,0.935,1.095,2019-12-09,0.955,6644600.0,0.915,0.970,17.112299,1
2,2020-03-10,1.500,1.290,2020-04-15,1.325,10114800.0,1.310,1.525,-14.000000,0
3,2020-09-11,3.000,3.340,2020-10-02,2.900,6414400.0,2.815,3.035,11.333333,1
4,2020-09-18,3.075,3.340,2020-10-02,3.000,937200.0,2.990,3.075,8.617886,1
5,2020-09-23,3.010,3.340,2020-10-02,2.975,2954000.0,2.935,3.030,10.963455,1
6,2020-12-07,4.250,4.650,2020-12-28,4.200,2806800.0,4.200,4.340,9.411765,1
7,2020-12-15,4.130,4.650,2020-12-28,4.040,2804800.0,3.965,4.190,12.590799,1
8,2020-12-18,4.165,4.650,2020-12-28,4.145,939600.0,4.105,4.195,11.644658,1
